In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import seaborn as sns
from PIL import Image
import cv2
import skimage.io
import skimage.transform
import matplotlib.pyplot as plt
from numpy import asarray
import random as rn
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import GlobalAveragePooling2D,Input, ReLU, Dense, Activation, ZeroPadding2D, Dropout,BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPool2D, Add, MaxPooling2D
from keras.models import Model, load_model, Sequential
from keras.models import save_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau 
from keras.initializers import glorot_uniform
from keras.applications.resnet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras import utils as np_utils
import keras.backend as K
from sklearn.utils import class_weight
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import train_test_split
########################## librerias para el desbalance de datos #######################
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
########################################################################################

Creación de un directorio

In [ ]:
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

Preprocesamiento de las imágenes

In [ ]:
data_path = "/home/joshua6090/Documentos/DL_AECG_P/anomalias_en_electrocardiogramas_si/kunst/PosibleProp/Papers/spectrogram_ecg_cnn/RP_plt"
dir_list=os.listdir(data_path)
print('Tu puta lista de archivos:',dir_list)
label=['N','Q','S','V','F']
N=[]
Q=[]
S=[]
V=[]
F=[]

for name_dir in dir_list:
    dir_path=data_path + '/' + name_dir
    print(dir_path) #### detecta las carpetas de las clases que tenemos
    for file_name in os.listdir(dir_path):
        image_path=dir_path + '/' + file_name #### tengo todas las dirrecciones de las imagenes
        checar= name_dir
        if checar == 'N':
            N.append(image_path)
        elif checar== 'Q':
            Q.append(image_path)
        elif checar== 'S':
            S.append(image_path)
        elif checar== 'V':
            V.append(image_path)
        else:
            F.append(image_path)

print('Hay {} N'.format(len(N)))
print('Hay {} Q'.format(len(Q)))
print('Hay {} S'.format(len(S)))
print('Hay {} V'.format(len(V)))
print('Hay {} F'.format(len(F)))



In [ ]:
from sklearn.preprocessing import  LabelEncoder 
plt.rcParams.update({'font.size': 16})
#### ramdom no seas wey, solo es eso
np.random.shuffle(N)
np.random.shuffle(Q)
np.random.shuffle(S)
np.random.shuffle(V)
np.random.shuffle(F)

#####################
#### split 

train_N, test_N = np.split(N, [int(len(N)*0.8)])
train_Q, test_Q = np.split(Q, [int(len(Q)*0.8)])
train_S, test_S = np.split(S, [int(len(S)*0.8)])
train_V, test_V = np.split(V, [int(len(V)*0.8)])
train_F, test_F = np.split(F, [int(len(F)*0.8)])

### DataFrames
train_N_df = pd.DataFrame({'image':train_N, 'label':'N'})
test_N_df = pd.DataFrame({'image':test_N, 'label':'N'})

train_Q_df = pd.DataFrame({'image':train_Q, 'label':'Q'})
test_Q_df = pd.DataFrame({'image':test_Q, 'label':'Q'})

train_S_df = pd.DataFrame({'image':train_S, 'label':'S'})
test_S_df = pd.DataFrame({'image':test_S, 'label':'S'})

train_V_df = pd.DataFrame({'image':train_V, 'label':'V'})
test_V_df = pd.DataFrame({'image':test_V, 'label':'V'})

train_F_df = pd.DataFrame({'image':train_F, 'label':'F'})
test_F_df = pd.DataFrame({'image':test_F, 'label':'F'})
### Juntar estas madres

train_df = pd.concat([train_N_df, train_Q_df, train_S_df , train_V_df, train_F_df])
test_df = pd.concat([test_N_df, test_Q_df, test_S_df, test_V_df, test_F_df])
#####################
#para  como se ve la distribución de los datos
todo_N = pd.DataFrame({'image':N, 'label':'N'})
todo_Q = pd.DataFrame({'image':Q, 'label':'Q'})
todo_S = pd.DataFrame({'image':S, 'label':'S'})
todo_V = pd.DataFrame({'image':V, 'label':'V'})
todo_F = pd.DataFrame({'image':F, 'label':'F'})
todo = pd.concat([todo_N,todo_Q,todo_S,todo_V,todo_F])

X=np.array(todo["image"].tolist())
X=X.reshape(-1,1)
y=np.array(todo["label"].tolist())
y.reshape(-1,1)
###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
w2=pd.DataFrame({'image':X.flatten(), 'label':y})
X_train_todo,y_test_todo = train_test_split(w2, test_size=0.2)
###############
#le=LabelEncoder()
#y=le.fit_transform(y)
##primero a datframe
#w=pd.DataFrame({'image':x})
#w1=pd.DataFrame({'label':y})
### junto
#xy=pd.concat([w,w1])
#####################
### Vamo a visualizar la distribución
fig=sns.countplot(x='label',data=todo)
plt.xlabel("Categorías")
plt.ylabel("Número de muestras")
plt.title("Distribución de la base de datos antes del RUS") 
plt.show(fig)



In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 16})
# Calcular el conteo de cada categoría
categories = todo['label'].value_counts()

# Definir los datos para el diagrama de torta
labels = categories.index
sizes = categories.values
explode = (0, 0.1, 0.3, 0.5,0.9)  # No se hará "explode" en ninguna categoría

# Crear el diagrama de torta y ajustar el tamaño de la fuente de las etiquetas
plt.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', textprops={'fontsize': 20})

# Configurar el aspecto del gráfico
plt.title("MIT-BIH")
plt.axis('equal')  # Para que el gráfico sea un círculo en lugar de una elipse

# Mostrar el gráfico
plt.show()


Problema de des-balanceamiento

In [ ]:
###### Primero ocuparemos el método de RandomUnderSampling
rus=RandomUnderSampler(sampling_strategy={'N':802,'Q':802,'S':802,'V':802,'F':802})
X_train_rus,y_test_rus=rus.fit_resample(X,y)
###Para ver el número de records
print(sorted(Counter(y_test_rus).items()))

###Concatnarlos en un data frame
#X_train_rus=np.array(X_train_rus.tolist())
w=pd.DataFrame({'image':X_train_rus.flatten(), 'label':y_test_rus})
#### como quedo despues del RUS
fig=sns.countplot(x='label',data=w)
plt.xlabel("Categorías")
plt.ylabel("Número de muestras")
plt.title("Distribución de la base de datos después del RUS") 
plt.show(fig)
###########################
X_train,y_test = train_test_split(w, test_size=0.2)
XX_Train=X_train.image
yy_Test=y_test.label

In [ ]:
import matplotlib.pyplot as plt

# Calcular el conteo de cada categoría después del RUS
categories_rus = w['label'].value_counts()

# Crear el gráfico de pastel
plt.pie(categories_rus, labels=categories_rus.index, autopct='%1.1f%%', textprops={'fontsize': 20})

# Configurar el aspecto del gráfico
plt.title("Distribución de la base de datos después del RUS.")
plt.axis('equal')

# Mostrar el gráfico
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Calcular el conteo de cada categoría
categories = todo['label'].value_counts()

# Crear el gráfico de pastel
plt.pie(categories, labels=categories.index, autopct='')

# Configurar el aspecto del gráfico
plt.title("MIT-BIH")
plt.axis('equal')  # Para que el gráfico sea un círculo en lugar de una elipse

# Calcular el centro del gráfico de pastel
x, y = 0, 0

# Calcular el radio del gráfico de pastel
radius = 0.5

# Mostrar los porcentajes fuera del gráfico
total = sum(categories)
previous_angle = 0

for category in categories.index:
    count = categories[category]
    percentage = count / total * 100

    # Calcular el ángulo correspondiente al segmento de cada categoría
    angle = 2 * 3.14159 * count / total + previous_angle

    # Calcular las coordenadas del punto de anotación
    x_anno = x + (radius + 0.1) * 0.5 * np.cos(angle)
    y_anno = y + (radius + 0.1) * 0.5 * np.sin(angle)

    # Mostrar el porcentaje como anotación
    plt.annotate(f'{percentage:.1f}%', (x_anno, y_anno), ha='center', va='center', fontsize=10)

    previous_angle = angle

# Mostrar el gráfico
plt.show()


In [ ]:
X_train.to_csv('X_train.csv', header=False, index=False)
y_test.to_csv('y_test.csv', header=False, index=False)

In [ ]:
###Empieza lo chido mi prro
batch_size = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224

#create the ImageDataGenerator object and rescale the images
trainGenerator = ImageDataGenerator(preprocessing_function=preprocess_input)
testGenerator = ImageDataGenerator(preprocessing_function=preprocess_input)

#convert them into a dataset
trainDataset = trainGenerator.flow_from_dataframe(
  dataframe=X_train_todo,
  class_mode='categorical',
  x_col="image",
  y_col="label",
  batch_size=batch_size,
  seed=42,
  shuffle=True,
  target_size=(IMG_HEIGHT,IMG_WIDTH) 
)


testDataset = testGenerator.flow_from_dataframe(
  dataframe=y_test_todo,
  class_mode='categorical',
  x_col="image",
  y_col="label",
  batch_size=batch_size,
  seed=42,
  shuffle=False,
  target_size=(IMG_HEIGHT,IMG_WIDTH)
)
completo=trainDataset.filenames
nb_train_samples = len(trainDataset.filenames)
nb_validation_samples = len(trainDataset.filenames)
Hola=testDataset.class_indices

RESNET 18

In [ ]:
INIT_SCHEME = "he_normal"
def res_block(X, filter, stage):
      
    # Convolutional_block
    X_copy = X

    f1 , f2, f3 = filter
    
    # Main Path
    X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_conv_a', kernel_initializer= INIT_SCHEME)(X)
    X = MaxPool2D((2,2))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_a')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_conv_b', kernel_initializer= INIT_SCHEME)(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_b')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_c', kernel_initializer= INIT_SCHEME)(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_c')(X)


    # Short path
    X_copy = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_copy', kernel_initializer= INIT_SCHEME)(X_copy)
    X_copy = MaxPool2D((2,2))(X_copy)
    X_copy = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_copy')(X_copy)

    # ADD
    X = Add()([X, X_copy])
    X = Activation('relu')(X)

    # Identity Block 1
    X_copy = X
    

    # Main Path
    X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_1_a', kernel_initializer= INIT_SCHEME)(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_a')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_1_b', kernel_initializer= INIT_SCHEME)(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_b')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_1_c', kernel_initializer= INIT_SCHEME)(X)
    X = BatchNormalization(axis = 3, name = 'bn_'+str(stage)+'_identity_1_c')(X)

    # ADD
    X = Add()([X, X_copy])
    X = Activation('relu')(X)

    # Identity Block 2
    X_copy = X


    # Main Path
    X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_2_a', kernel_initializer= INIT_SCHEME)(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_a')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_2_b', kernel_initializer= INIT_SCHEME)(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_b')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_2_c', kernel_initializer= INIT_SCHEME)(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_c')(X)

    # ADD
    X = Add()([X, X_copy])
    X = Activation('relu')(X)

    return X


input_shape = (224,224,3)

# Input tensor shape
X_input = Input(input_shape)

# Zero-padding
X = ZeroPadding2D((3,3))(X_input)

# 1- stage
X = Conv2D(64, (7,7), strides= (2,2), name = 'conv1', kernel_initializer= INIT_SCHEME)(X)
X = BatchNormalization(axis =3, name = 'bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3,3), strides= (2,2))(X)

# 2- stage
X = res_block(X, filter= [64,64,256], stage= 2)

# 3- stage
X = res_block(X, filter= [128,128,512], stage= 3)

# 4- stage
X = res_block(X, filter= [256,256,1024], stage= 4)

# 5- stage
X = res_block(X, filter= [512,512,2048], stage= 5)

# Average Pooling
X = AveragePooling2D((2,2), name = 'Averagea_Pooling')(X)

# Final layer
X = Flatten()(X)
X = Dropout(0.8)(X)
X = Dense(5, activation = 'softmax', name = 'Dense_final', kernel_initializer= INIT_SCHEME )(X)

# Build model.
model = Model(
    inputs= X_input, 
    outputs = X, 
    name = 'Resnet18'
    
)

# Check out model summary.
#model.summary()


Para compilar

In [ ]:
from gc import callbacks
import timeit
from tracemalloc import start
import h5py
import pickle as pkl
import json
from keras.models import save_model
epochs=100

optimizer =SGD(lr=1e-2)

model.compile(optimizer=optimizer,
            loss='categorical_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.8, patience=2,
                              verbose=1, mode='max', min_lr=1e-10)

callbacks=[reduce_lr]

history=model.fit(trainDataset,epochs=epochs ,validation_data=testDataset,batch_size=batch_size,
                                callbacks=callbacks,shuffle=True,
                                #class_weight=class_weights,
                                verbose=1)
dir='/home/joshua6090/Documentos/DL_AECG_P/anomalias_en_electrocardiogramas_si/kunst/PosibleProp/pesos/'

model.save('mmodel',save_format='keras')
model.save('model/')
#vamo a gusardar los pesos de esta madre.
model.save_weights(filepath= 'pesos.h5')
model.save("mi_modelo.h5")



Curvas de entrenamiento

In [ ]:
plt.rcParams.update({'font.size': 16})
plt.figure(0)  
plt.plot(history.history['accuracy'],'r')  
plt.plot(history.history['val_accuracy'],'g')  
plt.xticks(np.arange(0, 100, 10)) 
plt.yticks(np.arange(0, 1, 0.1))  
plt.rcParams['figure.figsize'] = (8, 6)  
plt.xlabel("Épocas")  
plt.ylabel("Accuracy")  
plt.title("Accuracy del entramiento y validación")  
plt.legend(['Entrenamiento','Validación'])

plt.figure(1)  
plt.plot(history.history['loss'],'r')  
plt.plot(history.history['val_loss'],'g')  
plt.xticks(np.arange(0, 100, 10))
   
plt.rcParams['figure.figsize'] = (8, 6)  
plt.xlabel("Épocas")  
plt.ylabel("Perdida")  
plt.title("Perdida del entramiento y validación")  
plt.legend(['Entrenamiento','Validación'])
plt.show()

Matriz de confusión.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn import metrics
#from sklearn.metrics import plot_confusion_matrix
#####################################################################
target_names=['N', 'Q','S','V','F']
plt.rcParams.update({'font.size': 16})
#####################################################################
errors=0
y_pred=[]
y_true=testDataset.labels # make sure shuffle=False in generator
classes=list(testDataset.class_indices.keys())
class_count=len(classes)
preds=model.predict(testDataset, verbose=1)
for i, p in enumerate(preds):        
        pred_index=np.argmax(p)         
        true_index=testDataset.labels[i]  # labels are integer values        
        if pred_index != true_index: # a misclassification has occurred                                           
            errors=errors + 1
        y_pred.append(pred_index)
        
tests=len(preds)
acc=( 1-errors/tests) * 100
msg=f'there were {errors} errors in {tests} tests for an accuracy of {acc:6.2f}'
print(msg)
#######################################################################
ypred=np.array(y_pred)
ytrue=np.array(y_true)
##############################################################################
cm = confusion_matrix(ytrue, ypred, normalize='true' )
plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False)       
plt.xticks(np.arange(class_count), classes, rotation=0)
plt.yticks(np.arange(class_count), classes, rotation=0)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
##################################################################################
clr = classification_report(y_true, y_pred, target_names=classes, digits= 3) # create classification report
print("Classification Report:\n----------------------\n", clr)
##################################################################################
#Bonita
cm_display=ConfusionMatrixDisplay(cm,display_labels=['F','N', 'Q','S','V'])
#print(cm)
cm_display.plot(cmap='Blues') 
cm_display.ax_.set_title('Matriz de confusión')



In [ ]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_true, y_pred, target_names=target_names, digits=3))

In [ ]:
per_class_accuracies = {}
for idx, cls in enumerate(classes):
    true_negatives = np.sum(np.delete(np.delete(cm, idx, axis=0), idx, axis=1))
    true_positives = cm[idx, idx]
    per_class_accuracies[cls] = (true_positives + true_negatives) / np.sum(cm)

ROC-AUC Curve

In [ ]:
predxs=model.predict(testDataset)
labelroc=testDataset

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, roc_auc_score, auc
plt.rcParams.update({'font.size': 16})
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    all_labels = ['N', 'Q', 'S', 'V', 'F']
    fig, c_ax = plt.subplots(1,1)
    for (idx, c_label) in enumerate(all_labels):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.3f)' % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'b-', label = 'Random Guessing',lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    c_ax.set_xlabel('Tasa de falsos positivos')
    c_ax.set_ylabel('Tasa de verdaderos positivos')
    c_ax.set_title('Característica Operativa del receptor')
    c_ax.legend(loc='lower right')
    plt.show()
    return roc_auc_score(y_test, y_pred, average=average)

# calling
testDataset.reset() # resetting generator
y_pred = model.predict_generator(testDataset, verbose = True)
y_pred = np.argmax(y_pred, axis=1)
multiclass_roc_auc_score(testDataset.classes, y_pred)